In [ ]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup
from datetime import datetime

In [ ]:
#Setup everything & open the browser

profile_path = r'C:\Users\<your_user>\AppData\Roaming\Mozilla\Firefox\Profiles\<profile_id>'
options=Options()
options.set_preference('profile', profile_path)
service = Service(r'driver/geckodriver.exe')

driver = Firefox(service=service, options=options)

driver.get("https://www.leagueofgraphs.com/summoner/euw/<Account pseudo>")

In [ ]:
#Click the "see more" matches recursively, click on the "I don't care about cookies" button if needed and delete the table header
while True:
    try:
        button = driver.find_element(By.CLASS_NAME, "see_more_ajax_button")
        button.click()
        time.sleep(0.5)
    except:
        break
try:
    driver.find_elements(By.CLASS_NAME, "ncmp__btn-text")[0].click()
except:
    print('Is the consent box already closed ? \'cause I couldn\'t find it ')
driver.execute_script("""
var element = arguments[0];
element.parentNode.removeChild(element);
""", driver.find_element(By.CLASS_NAME,'recentGamesTableHeader'))
driver.execute_script("""
var element = arguments[0];
element.parentNode.removeChild(element);
""", driver.find_element(By.CLASS_NAME,'recentGamesTableHeader'))

In [ ]:
table = driver.find_element(By.CLASS_NAME, "recentGamesTable")
soup = BeautifulSoup(table.get_attribute('outerHTML'))
last_cells = table.find_elements(By.CLASS_NAME, "summonersTdLight") #Find the champions table for each match
seeMatchButtons = [] #Get the see match button containing player id (which bug sometimes) and match id
#champsNames = [] #Get champion name to make sure that the id for that match is not broken
for cell in last_cells:
    try:
        seeMatchButtons.append(cell.find_element(By.CLASS_NAME, "seeMatchButton"))
        #champsNames.append(cell.find_element(By.CLASS_NAME, "selected").find_element(By.TAG_NAME, "img").get_attribute("alt"))
    except:
        pass

In [ ]:
i = 0
matches=list()
for match in soup.findAll("tr"): #For each match in the list
    columns = {}
    try:
        columns["button"] = (seeMatchButtons[i]) #Match button that will be clicked to get the details
         #columns.append(champsNames[i]) #Get champion name
        columns["id"] = (seeMatchButtons[i].get_attribute('href').split('#participant')[1]) #Get participant number
    except:
        pass
    gameMode = match.findAll("div", {"class": "gameMode"}) #Get match game mode
    victoryDefeat = match.findAll("div", {"class": "victoryDefeatText"}) #Get match status (victory/defeat)
    #Check that the match is not a remake or whatever else, and that it's a soloqueue so that 2 of the same champs can't be used in the same game
    if gameMode[0].text.strip() == "Soloqueue" and (victoryDefeat[0].text == "Victory" or victoryDefeat[0].text == "Defeat"):
        #columns.append(gameMode[0].text)
        for champcell in match.findAll("td", {"class": "championCellLight"}):
            for img in champcell.findAll('img', title=True):
                columns["champ"] = (img.get('title')) #Get champion name
        for kda in match.findAll("div", {"class": "kda"}):
            for kills in kda.findAll('span', {"class": "kills"}):
                columns["kills"] = kills.text
            for deaths in kda.findAll('span', {"class": "deaths"}):
                columns["deaths"] = deaths.text
            for assists in kda.findAll('span', {"class": "assists"}):
                columns["assists"] = assists.text
        columns["status"] = ('Win' if victoryDefeat[0].text == "Victory" else 'Lose')
        matches.append(columns)
    i+=1

In [ ]:
i = 0
for match in matches:
    driver.switch_to.window(driver.window_handles[0]) #Go to the navigator first tab
    match["button"].send_keys(Keys.CONTROL + Keys.RETURN) #Press ctrl+enter to open match in new tab
    driver.switch_to.window(driver.window_handles[1]) #Go to the navigator first tab

    for attempt in range(10): #Used to auto reload page and retry if something failed in case there is a network error or whatever
        try:
            #Wait until the page is fully loaded
            while True:
                time.sleep(0.5)
                if driver.execute_script('return document.readyState;') == 'complete':
                    break

            #Delete the ncmp__tool box if it exists
            try:
                ncmp = driver.find_element(By.ID, "ncmp__tool")
                driver.execute_script("""
                var element = arguments[0];
                element.parentNode.removeChild(element);
                """, ncmp)
                time.sleep(0.5)
            except:
                pass

            #time.sleep(0.5)
            #Get timestamp
            timestamp = int(driver.find_element(By.CLASS_NAME, "bannerSubtitle").find_element(By.TAG_NAME, "span").get_attribute("data-timestamp"))
            date = datetime.fromtimestamp(timestamp/1000) #Convert timestamp to date
            matches[i]["date"] = (str(date.day)+"/"+str(date.month)+"/"+str(date.year)) #Format match date


            driver.find_element(By.CLASS_NAME, "recapTableTab").click()

            #time.sleep(0.5)

            matchDetailsTable = BeautifulSoup(driver.find_element(By.ID, "match_datatable").get_attribute('outerHTML')) #Get the match info table

            matchTableHeader = driver.find_element(By.CLASS_NAME, "matchTable").find_element(By.TAG_NAME, "tr")
            #header_numbers = matchTableHeader.text.replace("Defeat", "").replace("Victory", "").replace("Ranked Solo/Duo (", "") \
            #                    .replace(")", "").replace("/ ", "").replace(":", " ").strip().split(' ')
            gameDurationMin = int(matchTableHeader.find_element(By.CLASS_NAME, "gameDuration").text.replace("(", "").replace(")", "").split(":")[0])

            #time.sleep(0.5)

            matchDetails=list()
            for detailRow in matchDetailsTable.findAll("tr"):
                columns = list()
                for column in detailRow.findAll("th"):
                    columns.append(column)
                for column in detailRow.findAll("td"):
                    columns.append(column)
                matchDetails.append(columns)

            #Make sure that the player ID is right manually
            j = 0
            player_id = 0
            for champ in matchDetails[0][0:]:
                if match["champ"] in str(champ):
                    player_id = j
                j+=1


            matches[i]["wards"] = (int(matchDetails[25][player_id].text.strip())) #Ward
            matches[i]["pinks"] = (int(matchDetails[27][player_id].text.strip())) #Pink
            matches[i]["vision score"] = (int(matchDetails[24][player_id].text.strip())) #Vision Score (VS)
            matches[i]["farm"] = (int(matchDetails[31][player_id].text.strip()) + int(matchDetails[32][player_id].text.strip())) #Minions
            matches[i]["duration"] = gameDurationMin  #Match duration
            matches[i]["total kills"] = (int(matchTableHeader.find_element(By.CLASS_NAME, "kda-left").text.split("/")[0]) if int(player_id) <= 5 else int(matchTableHeader.find_element(By.CLASS_NAME, "kda-right").text.split("/")[0]))
            matches[i]["total deaths"] = (int(matchTableHeader.find_element(By.CLASS_NAME, "kda-left").text.split("/")[1]) if int(player_id) <= 5 else int(matchTableHeader.find_element(By.CLASS_NAME, "kda-right").text.split("/")[1]))

            driver.close()
            driver.switch_to.window(driver.window_handles[0])

            i+=1
            break
        except Exception as e:
            print(e)
            driver.refresh()
            time.sleep(1)
            continue
        else:
            exit()
driver.quit()

In [ ]:
pd.DataFrame(matches).to_csv('stats.csv', index=False, sep=";", columns=["date", "champ", "kills", "deaths", "assists", "status", "wards", "pinks", "vision score", "farm", "duration", "total kills", "total deaths"])